In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sympy
from scipy import *
from scipy import integrate
from scipy.integrate import ode

In [2]:
#Importa Parametros de control

f = open ('parrov_bluerov.txt','r')
D={}
for i in f:
    [clave, valor]= i.split("=")
    D[clave] = float(valor[:-2])
f.close()
#Se asignan las variables... esto se hace para que sea sencillo describir mas adelante las matrices .... Ya estan los C+ y C- ??
[m,xg,yg,zg,xb,yb,zb,Ix,Iy,Iz,Ixy,Ixz,Iyz,W,B,Xu,Yv,Zw,Kp,Mq,Nr,Xudot,Yvdot,Zwdot,Kpdot,Mqdot,Nrdot,Xuu,Yvv,Zww,Kpp,Mqq,Nrr,lx1,ly1,lz1,lx2,ly2,lz2,lx3,ly3,lz3,lx4,ly4,lz4,lx5,ly5,lz5,lx6,ly6,lz6,Cmas1,Cmas2,Cmas3,Cmas4,Cmas5,Cmas6,Cmenos1,Cmenos2,Cmenos3,Cmenos4,Cmenos5,Cmenos6 ]   =   [D['m'],D['xg'],D['yg'],D['zg'],D['xb'],D['yb'],D['zb'],D['Ix'],D['Iy'],D['Iz'],D['Ixy'],D['Ixz'],D['Iyz'],D['W'],D['B'],D['Xu'],D['Yv'],D['Zw'],D['Kp'],D['Mq'],D['Nr'],D['Xudot'],D['Yvdot'],D['Zwdot'],D['Kpdot'],D['Mqdot'],D['Nrdot'],D['Xuu'],D['Yvv'],D['Zww'],D['Kpp'],D['Mqq'],D['Nrr'],D['lx1'],D['ly1'],D['lz1'],D['lx2'],D['ly2'],D['lz2'],D['lx3'],D['ly3'],D['lz3'],D['lx4'],D['ly4'],D['lz4'],D['lx5'],D['ly5'],D['lz5'],D['lx6'],D['ly6'],D['lz6'],0.000333,0.000333,0.000224,0.000241,0.000241,0.000243,0.000368,0.000368,0.000227,0.000175,0.000175,0.000229]




In [50]:
#Se crean las variables simbolicas dependientes e independientes

#Entradas para el modelo dinamico .... Torques aplicados por los propulsores
u1 = sympy.Symbol('u1')
u2 = sympy.Symbol('u2')
u3 = sympy.Symbol('u3')
u4 = sympy.Symbol('u4')
u5 = sympy.Symbol('u5')
u6 = sympy.Symbol('u6')
t = sympy.symbols("t", positive=True) #Tiempo
U= np.array([u1,u2,u3,u4,u5,u6]) # Se define el vector de entradas

#Salidas de mi modelo dinamico... o velocidades en el marco de referencia local
u = sympy.Function('u')
v = sympy.Function('v')
w = sympy.Function('w')
p = sympy.Function('p')
q = sympy.Function('q')
r = sympy.Function('r')
V = np.array([u,v,w,p,q,r])
Vdot = [] #Vector aceleracion en el marco de referencia
for i in V:
    Vdot.append(i(t).diff(t))

x = sympy.Function('u')
y = sympy.Function('v')
z = sympy.Function('w')
theta = sympy.Function('theta')
phi = sympy.Function('phi')
psi = sympy.Function('psi')
n = np.array([x,y,z,theta,phi,psi])

In [4]:
#Matriz M
#Matriz M_RB                     
M_RB = np.array([[     m        ,        0      ,        0      ,      0        ,     m*zg     ,       -m*yg    ],
                 [     0        ,        m      ,        0      ,    -m*zg      ,       0      ,        m*xg    ],
                 [     0        ,        0      ,        m      ,     m*yg      ,    -m*xg     ,          0     ],
                 [     0        ,      -m*zg    ,       m*yg    ,     -Ix       ,    -Ixy      ,        -Ixz    ],
                 [    m*zg      ,        0      ,      -m*xg    ,     -Ixy      ,     Iy       ,        -Iyz    ],
                 [   -m*yg      ,       m*xg    ,        0      ,     -Ixz      ,     -Iyz     ,         Iz     ]])
#Matriz M_A
M_A = np.array([Xudot,Yvdot,Zwdot,Kpdot,Mqdot,Nrdot]) * np.eye(6) # MUltiplica vector valores con matris identidad 6x6

M= M_RB + M_A

print("M_RB = \n{}".format(M_RB))
print("M_A = \n{}".format(M_A))
print("M = \n{}".format(M))

M_RB = 
[[ 7.31  0.    0.    0.    0.   -0.  ]
 [ 0.    7.31  0.   -0.    0.    0.  ]
 [ 0.    0.    7.31  0.   -0.    0.  ]
 [ 0.   -0.    0.   -0.16 -0.   -0.  ]
 [ 0.    0.   -0.   -0.    0.16 -0.  ]
 [-0.    0.    0.   -0.   -0.    0.16]]
M_A = 
[[ -5.5   -0.    -0.    -0.    -0.    -0.  ]
 [ -0.   -12.7   -0.    -0.    -0.    -0.  ]
 [ -0.    -0.   -14.57  -0.    -0.    -0.  ]
 [ -0.    -0.    -0.    -0.12  -0.    -0.  ]
 [ -0.    -0.    -0.    -0.    -0.12  -0.  ]
 [ -0.    -0.    -0.    -0.    -0.    -0.12]]
M = 
[[ 1.81  0.    0.    0.    0.   -0.  ]
 [ 0.   -5.39  0.   -0.    0.    0.  ]
 [ 0.    0.   -7.26  0.   -0.    0.  ]
 [ 0.   -0.    0.   -0.28 -0.   -0.  ]
 [ 0.    0.   -0.   -0.    0.04 -0.  ]
 [-0.    0.    0.   -0.   -0.    0.04]]


In [41]:
#Matriz C.... Matriz de fuerzas centrípeta y Coriolis
#Matriz C_RB
#Matriz C12 
C11 = np.eye(3)*0 #Crea matriz 3x3 de Ceros
C12 = np.array([[ m*(yg*q(t) +zg*r(t)), -m*(xg*q(t)-w(t))  , -m*(xg*r(t)+v(t))  ],
                [ -m*(yg*p(t)+w(t))   , m*(zg*r(t)+xg*p(t)), -m*(yg*r(t)-u(t))  ],
                [ -m*(zg*p(t)-v(t))   , -m*(zg*q(t)+u(t))  , m*(xg*p(t)+yg*q(t))]])

C22 = np.array([[         0                 ,-Iyz*q(t)-Ixz*p(t)+Iz*r(t), Iyz*r(t)+Ixy*p(t)-Iy*q(t)],
                [ Iyz*q(t)+Ixz*p(t)-+Iz*r(t),          0               ,-Ixz*r(t)-Ixy*q(t)+Iy*p(t)],
                [-Iyz*r(t)-Ixy*p(t)+Iy*q(t) , Ixz*r(t)+Ixy*q(t)-Iy*p(t), 0                  ]])
C21 = np.transpose(C12) # Transpuesta de C12
C_RB = np.concatenate((np.concatenate((C11, C12), axis=1), np.concatenate((C21, C22), axis=1)), axis=0)



CA = np.array([[     0        ,        0      ,        0      ,      0        , -Zwdot*w(t)  ,     Yvdot*v(t) ],
               [     0        ,        0      ,        0      ,  Zwdot*w(t)   ,       0      ,    -Xudot*u(t) ],
               [     0        ,        0      ,        0      , -Yvdot*v(t)   ,  Xudot*u(t)  ,          0     ],
               [      0       , -Zwdot*w(t)   ,    Yvdot*v(t) ,      0        , -Nrdot*r(t)  ,     Mqdot*q(t) ],
               [  Zwdot*w(t)  ,        0      ,   -Xudot*u(t) ,  Nrdot*r(t)   ,       0      ,    -Kpdot*p(t) ],
               [ -Yvdot*v(t)  ,  Xudot*u(t)   ,          0    , -Mqdot*q(t)   ,  Kpdot*p(t)  ,          0     ]])

C = C_RB + CA
print("C_RB = \n{}".format(C_RB))
print("CA = \n{}".format(CA))
print("C = \n{}".format(C))


C_RB = 
[[0.0 0.0 0.0 0 7.31*w(t) -7.31*v(t)]
 [0.0 0.0 0.0 -7.31*w(t) 0 7.31*u(t)]
 [0.0 0.0 0.0 7.31*v(t) -7.31*u(t) 0]
 [0 -7.31*w(t) 7.31*v(t) 0 0.16*r(t) -0.16*q(t)]
 [7.31*w(t) 0 -7.31*u(t) -0.16*r(t) 0 0.16*p(t)]
 [-7.31*v(t) 7.31*u(t) 0 0.16*q(t) -0.16*p(t) 0]]
CA = 
[[0 0 0 0 14.57*w(t) -12.7*v(t)]
 [0 0 0 -14.57*w(t) 0 5.5*u(t)]
 [0 0 0 12.7*v(t) -5.5*u(t) 0]
 [0 14.57*w(t) -12.7*v(t) 0 0.12*r(t) -0.12*q(t)]
 [-14.57*w(t) 0 5.5*u(t) -0.12*r(t) 0 0.12*p(t)]
 [12.7*v(t) -5.5*u(t) 0 0.12*q(t) -0.12*p(t) 0]]
C = 
[[0.0 0.0 0.0 0 21.88*w(t) -20.01*v(t)]
 [0.0 0.0 0.0 -21.88*w(t) 0 12.81*u(t)]
 [0.0 0.0 0.0 20.01*v(t) -12.81*u(t) 0]
 [0 7.26*w(t) -5.39*v(t) 0 0.28*r(t) -0.28*q(t)]
 [-7.26*w(t) 0 -1.81*u(t) -0.28*r(t) 0 0.28*p(t)]
 [5.39*v(t) 1.81*u(t) 0 0.28*q(t) -0.28*p(t) 0]]


In [48]:
#Matriz de amortiguamiento hidrodinamico
DL = np.array([Xu,Yv,Zw,Kp,Mq,Nr]) * np.eye(6)
DQ = np.array([Xuu*u(t),Yvv*v(t),Zww*w(t),Kpp*p(t),Mqq*q(t),Nrr*r(t)]) * np.eye(6)
print("DL = \n{}".format(DL))
print("DQ = \n{}".format(DQ))


DL = 
[[-4.03 -0.   -0.   -0.   -0.   -0.  ]
 [-0.   -6.22 -0.   -0.   -0.   -0.  ]
 [-0.   -0.   -5.18 -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -0.07 -0.   -0.  ]
 [-0.   -0.   -0.   -0.   -0.07 -0.  ]
 [-0.   -0.   -0.   -0.   -0.   -0.07]]
DQ = 
[[-18.18*u(t) 0 0 0 0 0]
 [0 -21.66*v(t) 0 0 0 0]
 [0 0 -36.99*w(t) 0 0 0]
 [0 0 0 -1.55*p(t) 0 0]
 [0 0 0 0 -1.55*q(t) 0]
 [0 0 0 0 0 -1.55*r(t)]]


In [6]:
#Matriz de configuracion de los propulsores T
T= np.array([[     0        ,        0      ,        0      ,      1        ,       1     ,          0      ],
             [     0        ,        m      ,        0      ,      0        ,       0     ,          1      ],
             [    -1        ,        -1     ,        -1     ,      0        ,       0     ,          0      ],
             [   -ly1       ,      -ly2     ,      -ly3     ,      0        ,       0     ,        -lz6     ],
             [    lx1       ,        lx2    ,       lx3     ,     lz4       ,      lz5    ,          0      ],
             [     0        ,        0      ,        0      ,    -ly4       ,     -ly5    ,         lx6     ]])
Cmas = np.array([Cmas1,Cmas2,Cmas3,Cmas4,Cmas5,Cmas6]) * np.eye(6)
print("T = \n{}".format(T))
print("C+ = \n{}".format(Cmas))

T = 
[[ 0.00e+00  0.00e+00  0.00e+00  1.00e+00  1.00e+00  0.00e+00]
 [ 0.00e+00  7.31e+00  0.00e+00  0.00e+00  0.00e+00  1.00e+00]
 [-1.00e+00 -1.00e+00 -1.00e+00  0.00e+00  0.00e+00  0.00e+00]
 [ 1.11e-04 -1.11e-04 -0.00e+00  0.00e+00  0.00e+00 -9.50e-05]
 [ 1.53e-04  1.53e-04 -1.99e-04 -1.40e-05 -1.40e-05  0.00e+00]
 [ 0.00e+00  0.00e+00  0.00e+00 -1.11e-04  1.11e-04  8.00e-06]]
C+ = 
[[0.000333 0.       0.       0.       0.       0.      ]
 [0.       0.000333 0.       0.       0.       0.      ]
 [0.       0.       0.000224 0.       0.       0.      ]
 [0.       0.       0.       0.000241 0.       0.      ]
 [0.       0.       0.       0.       0.000241 0.      ]
 [0.       0.       0.       0.       0.       0.000243]]


In [7]:
f = np.dot(M,Vdot) #Multiplicacion de matriz vector
torques= np.dot(np.dot(T,Cmas),U)

sympy.dsolve(f[2]-torques[2])

Eq(w(t), C1 + t*(333*u1 + 333*u2 + 224*u3)/7260000)